In [4]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using GridInterpolations

using POMDPPolicies

using JLD2
using FileIO
using Reel
using Random
using POMDPs
using POMDPModelTools
using LinearAlgebra

using PedestrianAvoidancePOMDP
using EmergencyBrakingSystem

using CSV
using DataFrames

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1273


In [5]:
# Evaluate EuroNCAP scenarios

#algorithm = "EmergancyBrakingSystem"
#algorithm = "PedestrianAvoidancePOMDP"
algorithm = "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"

policy_name = "longitudinal"
#policy_name = "longitudinal_lateral_support"
policy = load("../policy/policy.jld2")["policy"];


log_filename = string("results_", algorithm, "_", policy_name, ".csv")
println(log_filename)

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA", "FP"]

vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = [50.]  # evalaute only on the scenarios with 50km/h
vut_speeds = vut_speeds / 3.6

probability_pedestrian_birth = policy.pomdp.PROBABILITY_PEDESTRIAN_BIRTH


println("Algorithm to evaluate: ", algorithm, " Policy: ", policy_name, " (if specified)")

# Evaluate all EuroNCAP scenarios
results = Vector[]
for scenario in scenarios
    println("Scenario: ", scenario)
    if (scenario == "FP") 
        hit_points = [-100., 200.]
    else
        if (scenario == "CPCN")
            hit_points = [0., 10., 20., 30., 40., 50., 800.] # additional scenario without crossing pedestrian
        else
            hit_points = [0., 10., 20., 30., 40., 50.]
        end
    end
    for hit_point in hit_points
        for vut_speed in vut_speeds
            ego_v = vut_speed
            (ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = PedestrianAvoidancePOMDP.generate_scenario(scenario, ego_v, hit_point)
            if algorithm == "EmergancyBrakingSystem"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
            elseif algorithm == "PedestrianAvoidancePOMDP"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy, algorithm, probability_pedestrian_birth, [0.0])
            elseif algorithm == "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy, algorithm, probability_pedestrian_birth, [0.0])
            else
                println("No valid algorithm defined!")
                return false
            end
            println(scenario, " HP: ", hit_point, " ", ego_x, " ", ego_y, " ", ego_v, " ", ped_x," ",  ped_y," ",  ped_v)
            (collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = PedestrianAvoidancePOMDP.evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision, ped_x)
            println("Collision: ", collision, " eb: ", emergency_brake_intervention, " ", dv_collision, " ", v_mean, " ", a_mean, " ", a_jerk, " ", a_min)
            result = [scenario_id, ego_v, hit_point, collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min  ]
            push!(results, result)
        end
    end 
end  


# store results from scneario evaluation in log file
df = DataFrame(results)
df = DataFrame(Matrix(df)')
rename!(df, :x1 => :scenario_id, :x2 => :ego_v, :x3 => :hit_point, :x4 => :collision, :x5 => :eb_intervention)
rename!(df, :x6 => :dv_collision, :x7 => :v_mean, :x8 => :a_mean, :x9 => :a_jerk, :x10 => :a_min)
CSV.write(log_filename, df);


(sum_collisions, sum_eb, dv, v_mean, a_mean, a_jerk, a_min) = PedestrianAvoidancePOMDP.evaluateScenariosMetric(results)
println("#collisions: ", sum_collisions, " #emergency brake interventions: ", sum_eb, " dv_m: ", dv, " v_mean: ", v_mean, " a_mean: ", a_mean, " a_jerk: ", a_jerk, " a_min: ", a_min)

df


results_PedestrianAvoidancePOMDP_EmergencyBrakingSystem_longitudinal.csv
Algorithm to evaluate: PedestrianAvoidancePOMDP_EmergencyBrakingSystem Policy: longitudinal (if specified)
Scenario: CPCN
PedestrianAvoidancePOMDP_EmergencyBrakingSystem
-----------------> Collision <----------------------
CPCN HP: 0.0 56.33333333333333 0.0 13.88888888888889 100.0 -5.066666666666666 1.3888888888888888
Collision: true eb: true 8.18888888888892 13.045238095238101 -4.371428571428571 103.0 -10.0
PedestrianAvoidancePOMDP_EmergencyBrakingSystem
-----------------> Collision <----------------------
CPCN HP: 10.0 56.33333333333333 0.0 13.88888888888889 100.0 -4.886666666666666 1.3888888888888888
Collision: true eb: true 4.188888888888918 12.469949494949498 -5.5 97.0 -10.0
PedestrianAvoidancePOMDP_EmergencyBrakingSystem
CPCN HP: 20.0 56.33333333333333 0.0 13.88888888888889 100.0 -4.706666666666666 1.3888888888888888
Collision: false eb: true 0.0 10.884429824561426 -6.303921568627451 125.0 -10.0
PedestrianAv

InterruptException: InterruptException: